In [40]:
import json
import os
import time
import random
import gradio as gr
import numpy as np
import requests
from PIL import Image
import matplotlib.pyplot as plt

In [50]:
URL = "http://127.0.0.1:8188/api/prompt"
OUTPUT_DIR = '/media/avidmech/data/ComfyUI/output'
INPUT_DIR = '/media/avidmech/data/ComfyUI/input'

In [42]:
def get_latest_image(folder):
    files = os.listdir(folder)
    image_files = [f for f in files if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
    image_files.sort(key=lambda x: os.path.getmtime(os.path.join(folder, x)))
    latest_image = os.path.join(folder, image_files[-1]) if image_files else None
    return latest_image


In [43]:
def start_queue(prompt_workflow):
    p = {"prompt": prompt_workflow}
    data = json.dumps(p).encode('utf-8')
    requests.post(URL, data=data)


In [48]:
def generate_image(prompt_text, step, cfg, input_image):   
    with open("Image Editing Qwen (1).json", "r") as file_json:
        prompt = json.load(file_json)
        prompt["5"]["inputs"]["prompt"] = f'{prompt_text}'
        prompt["12"]["inputs"]["steps"] = f'{step}'
        prompt["12"]["inputs"]["cfg"] = f'{cfg}'

    image = Image.fromarray(input_image)
    min_side = min(image.size)
    scale_factor = 512 / min_side
    new_size = (round(image.size[0] * scale_factor), round(image.size[1] * scale_factor))
    resized_image = image.resize(new_size)

    resized_image.save(os.path.join(INPUT_DIR, "test.jpg"))
    previous_image = get_latest_image(OUTPUT_DIR)
    
    start_queue(prompt)

    while True:
        latest_image = get_latest_image(OUTPUT_DIR)
        if latest_image != previous_image:
            return latest_image

        time.sleep(1)

In [49]:
demo = gr.Interface(fn = generate_image, inputs = ["text", "text", "text", 'image'], outputs = ['image'])
demo.launch()

* Running on local URL:  http://127.0.0.1:7869
* To create a public link, set `share=True` in `launch()`.
